In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

# Load the dataset (for example, faces)
# You can use datasets like CelebA, MNIST, CIFAR-10, etc.
# Here, we'll use MNIST for simplicity

(x_train, _), (_, _) = keras.datasets.mnist.load_data()

# Normalize the images
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)

# Set the dimensions of the random noise vector
latent_dim = 128

# Define the generator network
generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(7 * 7 * 128),
        layers.Reshape((7, 7, 128)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
        layers.Conv2DTranspose(1, kernel_size=7, strides=1, padding="same", activation="sigmoid"),
    ],
    name="generator"
)

# Define the discriminator network
discriminator = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, kernel_size=3, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator"
)

# Compile the discriminator
discriminator.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Compile the combined model (generator + discriminator)
discriminator.trainable = False
gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.Model(gan_input, gan_output)
gan.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Training loop
batch_size = 64
epochs = 100
save_interval = 10

for epoch in range(epochs):
    # Select a random batch of images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_images = x_train[idx]

    # Generate random noise as input to the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Generate fake images using the generator
    fake_images = generator.predict(noise)

    # Concatenate real and fake images into a single array
    combined_images = np.concatenate([real_images, fake_images])

    # Labels for real and fake images
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

    # Add random noise to the labels - important trick!
    labels += 0.05 * np.random.random(labels.shape)

    # Train the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # Generate noise for the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Labels for generated images (pretend they are real)
    misleading_targets = np.zeros((batch_size, 1))

    # Train the generator
    g_loss = gan.train_on_batch(noise, misleading_targets)

    # Print the progress and save images
    if epoch % save_interval == 0:
        print(f"Epoch {epoch}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

        # Save generated images
        row, col = 5, 5
        noise = np.random.normal(0, 1, (row * col, latent_dim))
        generated_images = generator.predict(noise)

        # Rescale images 0 - 1
        generated_images = 0.5 * generated_images + 0.5

        fig, axs = plt.subplots(row, col)
        idx = 0
        for i in range(row):
            for j in range(col):
                axs[i,j].imshow(generated_images[idx, :, :, 0], cmap='gray')
                axs[i,j].axis('off')
                idx += 1
        fig.savefig(f"generated_images_epoch_{epoch}.png")
        plt.close()


2/2 [==============================] - 0s 29ms/step
Epoch 0, Discriminator Loss: 0.6967575550079346, Generator Loss: 0.6905517578125
2/2 [==============================] - 0s 28ms/step
Epoch 10, Discriminator Loss: 0.6205289959907532, Generator Loss: 0.6820030212402344
2/2 [==============================] - 0s 28ms/step
Epoch 20, Discriminator Loss: 0.5508966445922852, Generator Loss: 0.6287754774093628
2/2 [==============================] - 0s 28ms/step
Epoch 30, Discriminator Loss: 0.46805423498153687, Generator Loss: 0.532321572303772
2/2 [==============================] - 0s 34ms/step
Epoch 40, Discriminator Loss: 0.35770535469055176, Generator Loss: 0.3380371034145355
2/2 [==============================] - 0s 29ms/step
Epoch 50, Discriminator Loss: 0.2458736002445221, Generator Loss: 0.24486622214317322
2/2 [==============================] - 0s 28ms/step
Epoch 60, Discriminator Loss: 0.15586860477924347, Generator Loss: 0.1687711924314499
2/2 [==============================] - 0s 